In [1]:
import findspark
findspark.init()

import requests
import json
import unidecode
from pyspark.sql import SparkSession
import pandas as pd
import time
spark = SparkSession.builder \
      .master("local[1]") \
      .appName("aprendendo") \
      .getOrCreate()

In [2]:
r = requests.get('https://servicodados.ibge.gov.br/api/v1/localidades/mesorregioes/3513/municipios')
responseCidade = r.json()

In [3]:
# Buscar cidades do Vale do Paraíba
# TODO
CodigoDaCidade = [responseCidade[c]['id'] for c in range (0,len(responseCidade))]
cidades = [responseCidade[c]['nome'] for c in range (0,len(responseCidade))]
id_microregiao = [responseCidade[c]['microrregiao']['id'] for c in range (0,len(responseCidade))]
nome_microregiao = [responseCidade[c]['microrregiao']['nome'] for c in range (0,len(responseCidade))]
id_mesorregiao = [responseCidade[c]['microrregiao']['mesorregiao']['id'] for c in range (0,len(responseCidade))]
nome_mesoregiao = [responseCidade[c]['microrregiao']['mesorregiao']['nome'] for c in range (0,len(responseCidade))]
id_UF_mesoregiao = [responseCidade[c]['microrregiao']['mesorregiao']['UF']['id'] for c in range (0,len(responseCidade))]
sigla_UF_mesoregiao = [responseCidade[c]['microrregiao']['mesorregiao']['UF']['sigla'] for c in range (0,len(responseCidade))]
nome_UF_mesoregiao = [responseCidade[c]['microrregiao']['mesorregiao']['UF']['nome'] for c in range (0,len(responseCidade))]
id_regiao_mesoregiao = [responseCidade[c]['microrregiao']['mesorregiao']['UF']['regiao']['id'] for c in range (0,len(responseCidade))]
sigla_regiao_mesoregiao = [responseCidade[c]['microrregiao']['mesorregiao']['UF']['regiao']['sigla'] for c in range (0,len(responseCidade))]
nome_regiao_mesoregiao = [responseCidade[c]['microrregiao']['mesorregiao']['UF']['regiao']['nome'] for c in range (0,len(responseCidade))]
id_reg_imediata = [responseCidade[c]['regiao-imediata']['id'] for c in range (0,len(responseCidade))]
nome_reg_imediata = [responseCidade[c]['regiao-imediata']['nome'] for c in range (0,len(responseCidade))]
id_regiao_intermediaria = [responseCidade[c]['regiao-imediata']['regiao-intermediaria']['id'] for c in range (0,len(responseCidade))]
nome_regiao_intermediaria = [responseCidade[c]['regiao-imediata']['regiao-intermediaria']['nome'] for c in range (0,len(responseCidade))]
id_uf_regiao_intermediaria = [responseCidade[c]['regiao-imediata']['regiao-intermediaria']['UF']['id'] for c in range (0,len(responseCidade))]
sigla_uf_regiao_intermediaria = [responseCidade[c]['regiao-imediata']['regiao-intermediaria']['UF']['sigla'] for c in range (0,len(responseCidade))]
nome_uf_regiao_intermediaria = [responseCidade[c]['regiao-imediata']['regiao-intermediaria']['UF']['nome'] for c in range (0,len(responseCidade))]
id_regiao_regiao_intermediarioa = [responseCidade[c]['regiao-imediata']['regiao-intermediaria']['UF']['regiao']['id'] for c in range (0,len(responseCidade))]
sigla_regiao_regiao_intermediarioa = [responseCidade[c]['regiao-imediata']['regiao-intermediaria']['UF']['regiao']['sigla'] for c in range (0,len(responseCidade))]
nome_regiao_regiao_intermediarioa = [responseCidade[c]['regiao-imediata']['regiao-intermediaria']['UF']['regiao']['nome'] for c in range (0,len(responseCidade))]

In [4]:
# Criar data frame com as cidades
# TODO

df_cidades = pd.DataFrame(list(zip(CodigoDaCidade,cidades,id_microregiao,nome_microregiao,id_mesorregiao,nome_mesoregiao,id_UF_mesoregiao,
                                   sigla_UF_mesoregiao,nome_UF_mesoregiao,id_regiao_mesoregiao,sigla_regiao_mesoregiao,nome_regiao_mesoregiao,
                                   id_reg_imediata,nome_reg_imediata,id_regiao_intermediaria,nome_regiao_intermediaria,id_uf_regiao_intermediaria,
                                   sigla_uf_regiao_intermediaria,nome_uf_regiao_intermediaria,id_regiao_regiao_intermediarioa,sigla_regiao_regiao_intermediarioa,
                                   nome_regiao_regiao_intermediarioa
                                  
                                  
                                  )),columns=
['CodigoDaCidade','cidades','id_microregiao','nome_microregiao','id_mesorregiao','nome_mesoregiao','id_UF_mesoregiao',
 'sigla_UF_mesoregiao','nome_UF_mesoregiao','id_regiao_mesoregiao','sigla_regiao_mesoregiao','nome_regiao_mesoregiao','id_reg_imediata','nome_reg_imediata',
'id_regiao_intermediaria','nome_regiao_intermediaria','id_uf_regiao_intermediaria','sigla_uf_regiao_intermediaria','nome_uf_regiao_intermediaria',
'id_regiao_regiao_intermediarioa','sigla_regiao_regiao_intermediarioa','nome_regiao_regiao_intermediarioa'])

In [5]:
# Criar view com as cidades
# TODO

df_spark_cidades = spark.createDataFrame(df_cidades)

try:
    df_spark_cidades.createTempView('cities')
except:
    print('já existe esta view')
    
cities = spark.sql("SELECT cidades,nome_microregiao,nome_mesoregiao,nome_regiao_mesoregiao,nome_reg_imediata,nome_regiao_intermediaria FROM cities")

In [6]:
cities = cities.toPandas()

In [7]:
cities.to_csv('view_cidades')

In [8]:
import unicodedata
import re

df_cidades['cidades']

def remove_non_ascii_normalized(string: str) -> str:
    normalized = unicodedata.normalize('NFD', string)
    return normalized.encode('ascii', 'ignore').decode('utf8').casefold()

cidades = df_cidades['cidades'].apply(remove_non_ascii_normalized)

In [9]:
# Buscar previsão do tempo para as cidades
# TODO

listaGeral = []
CidadeInvalidos = []
key = '1f5aac72c1f24090a5e154626222506'


#1 minuto para respeitar as novas requisições ao servidor do IBGE
time.sleep(60)


for index in range(0,len(cidades)):
    
    r = requests.get('https://servicodados.ibge.gov.br/api/v1/localidades/mesorregioes/3513/municipios')
    responseCidade = r.json()
    
    rqst = requests.get(f'http://api.weatherapi.com/v1/forecast.json?key={key}&q={cidades[index]}&days=5&aqi=no&alerts=no')
    response = rqst.json()
    
    for i in range(0,len(response)):
        
        if len(response.keys()) < 3:
            CidadeInvalidos.append(cidades[index])
        else:
            caixa = []
           
            caixa.append(responseCidade[index]['id']) 
            caixa.append(response['location']['name'])
            caixa.append(response['location']['region'])
            caixa.append(response['location']['country'])
            caixa.append(response['location']['lat'])
            caixa.append(response['location']['lon'])
            caixa.append(response['location']['tz_id'])
            caixa.append(response['location']['localtime_epoch'])
            caixa.append(response['location']['localtime'])
            caixa.append(response['current']['temp_f'])
            caixa.append('Sim' if response['current']['is_day'] == 1 else 'Não')
            caixa.append(response['current']['condition']['text'])
            caixa.append(response['current']['wind_kph'])
            caixa.append(response['current']['wind_degree'])
            caixa.append(response['current']['wind_dir'])
            caixa.append(response['current']['humidity'])
            caixa.append(response['current']['cloud'])
            caixa.append(response['current']['feelslike_c'])
            caixa.append(response['current']['vis_km'])
            caixa.append(response['current']['uv'])
            caixa.append(response['current']['gust_kph'])
            caixa.append(response['forecast']['forecastday'][i]['date'])
            caixa.append(response['forecast']['forecastday'][i]['day']['maxtemp_c'])
            caixa.append(response['forecast']['forecastday'][i]['day']['mintemp_c'])
            caixa.append(response['forecast']['forecastday'][i]['day']['avgtemp_c'])
            caixa.append('Sim' if response['forecast']['forecastday'][i]['day']['daily_will_it_rain'] == 1 else 'Não')
            caixa.append(response['forecast']['forecastday'][i]['day']['daily_chance_of_rain'])
            caixa.append(response['forecast']['forecastday'][i]['astro']['sunrise'])
            caixa.append(response['forecast']['forecastday'][i]['astro']['sunset'])
            caixa.append(response['forecast']['forecastday'][i]['day']['condition']['text'])
            caixa.append(response['forecast']['forecastday'][0]['day']['maxwind_kph'])
            listaGeral.append(caixa)       

In [10]:
# Criar data frame com as previsões
# TODO

df_forecast = pd.DataFrame(listaGeral,columns=['CodigoCidade','Cidade','Regiao','Pais','Latitude','Longitude','FusoHorario','EpocaLocal',
                                              'HorarioLocal','Temperatura','eDia','CondicaoDia','KmVelocidadeVento',
                                              'GrauVento','DirecaoVento','Humidade','CoberturaNuvensPercent','SensacaoTermina',
                                              'Visibilidade','RaiosUv','KmRajadaVento','Data','TemperaturaMax',
                                              'TemperaturaMin','TemperaturaMed','VaiChover','ProbChuvaPercent','NascerSol',
                                              'PorSol','CondicaoDoTempo','VelocidadeMaximaDoVento'])
df_forecast

,CodigoCidade,Cidade,Regiao,Pais,Latitude,Longitude,FusoHorario,EpocaLocal,HorarioLocal,Temperatura,...,Data,TemperaturaMax,TemperaturaMin,TemperaturaMed,VaiChover,ProbChuvaPercent,NascerSol,PorSol,CondicaoDoTempo,VelocidadeMaximaDoVento
0,3502507,Aparecida,Sao Paulo,Brazil,-22.83,-45.23,America/Sao_Paulo,1656513622,2022-06-29 11:40,75.2,...,2022-06-29,30.1,9.1,16.8,Não,0,06:42 AM,05:27 PM,Sunny,11.9
1,3502507,Aparecida,Sao Paulo,Brazil,-22.83,-45.23,America/Sao_Paulo,1656513622,2022-06-29 11:40,75.2,...,2022-06-30,26.4,7.3,14.8,Não,0,06:42 AM,05:28 PM,Sunny,11.9
2,3502507,Aparecida,Sao Paulo,Brazil,-22.83,-45.23,America/Sao_Paulo,1656513622,2022-06-29 11:40,75.2,...,2022-07-01,28.8,7.0,14.7,Não,0,06:42 AM,05:28 PM,Sunny,11.9
3,3503158,Arapei,Sao Paulo,Brazil,-22.68,-44.45,America/Sao_Paulo,1656513623,2022-06-29 11:40,69.1,...,2022-06-29,24.8,13.3,17.8,Não,0,06:38 AM,05:25 PM,Sunny,6.1
4,3503158,Arapei,Sao Paulo,Brazil,-22.68,-44.45,America/Sao_Paulo,1656513623,2022-06-29 11:40,69.1,...,2022-06-30,26.1,12.6,17.4,Não,0,06:38 AM,05:25 PM,Sunny,6.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109,3554805,Tremembe,Sao Paulo,Brazil,-22.97,-45.55,America/Sao_Paulo,1656513650,2022-06-29 11:40,73.4,...,2022-06-30,25.4,11.4,16.2,Não,0,06:44 AM,05:28 PM,Sunny,15.8
110,3554805,Tremembe,Sao Paulo,Brazil,-22.97,-45.55,America/Sao_Paulo,1656513650,2022-06-29 11:40,73.4,...,2022-07-01,28.6,10.2,16.5,Não,0,06:44 AM,05:29 PM,Sunny,15.8
111,3555406,Ubatuba,Sao Paulo,Brazil,-23.43,-45.07,America/Sao_Paulo,1656513650,2022-06-29 11:40,64.9,...,2022-06-29,25.4,12.2,17.0,Não,0,06:42 AM,05:26 PM,Sunny,12.2
112,3555406,Ubatuba,Sao Paulo,Brazil,-23.43,-45.07,America/Sao_Paulo,1656513650,2022-06-29 11:40,64.9,...,2022-06-30,22.7,10.8,15.2,Não,0,06:42 AM,05:26 PM,Sunny,12.2


In [11]:
# Criar view com as previsões
# TODO

df_spark_dataframe = spark.createDataFrame(df_forecast)

try:
    df_spark_dataframe.createTempView('dataframe_previsoes_visao_geral')
except:
    print('já existe esta view')
    
dataframe_previsoes_visao_geral = spark.sql("SELECT * from dataframe_previsoes_visao_geral" )

In [12]:
dataframe_previsoes_visao_geral.toPandas()

,CodigoCidade,Cidade,Regiao,Pais,Latitude,Longitude,FusoHorario,EpocaLocal,HorarioLocal,Temperatura,...,Data,TemperaturaMax,TemperaturaMin,TemperaturaMed,VaiChover,ProbChuvaPercent,NascerSol,PorSol,CondicaoDoTempo,VelocidadeMaximaDoVento
0,3502507,Aparecida,Sao Paulo,Brazil,-22.83,-45.23,America/Sao_Paulo,1656513622,2022-06-29 11:40,75.2,...,2022-06-29,30.1,9.1,16.8,Não,0,06:42 AM,05:27 PM,Sunny,11.9
1,3502507,Aparecida,Sao Paulo,Brazil,-22.83,-45.23,America/Sao_Paulo,1656513622,2022-06-29 11:40,75.2,...,2022-06-30,26.4,7.3,14.8,Não,0,06:42 AM,05:28 PM,Sunny,11.9
2,3502507,Aparecida,Sao Paulo,Brazil,-22.83,-45.23,America/Sao_Paulo,1656513622,2022-06-29 11:40,75.2,...,2022-07-01,28.8,7.0,14.7,Não,0,06:42 AM,05:28 PM,Sunny,11.9
3,3503158,Arapei,Sao Paulo,Brazil,-22.68,-44.45,America/Sao_Paulo,1656513623,2022-06-29 11:40,69.1,...,2022-06-29,24.8,13.3,17.8,Não,0,06:38 AM,05:25 PM,Sunny,6.1
4,3503158,Arapei,Sao Paulo,Brazil,-22.68,-44.45,America/Sao_Paulo,1656513623,2022-06-29 11:40,69.1,...,2022-06-30,26.1,12.6,17.4,Não,0,06:38 AM,05:25 PM,Sunny,6.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109,3554805,Tremembe,Sao Paulo,Brazil,-22.97,-45.55,America/Sao_Paulo,1656513650,2022-06-29 11:40,73.4,...,2022-06-30,25.4,11.4,16.2,Não,0,06:44 AM,05:28 PM,Sunny,15.8
110,3554805,Tremembe,Sao Paulo,Brazil,-22.97,-45.55,America/Sao_Paulo,1656513650,2022-06-29 11:40,73.4,...,2022-07-01,28.6,10.2,16.5,Não,0,06:44 AM,05:29 PM,Sunny,15.8
111,3555406,Ubatuba,Sao Paulo,Brazil,-23.43,-45.07,America/Sao_Paulo,1656513650,2022-06-29 11:40,64.9,...,2022-06-29,25.4,12.2,17.0,Não,0,06:42 AM,05:26 PM,Sunny,12.2
112,3555406,Ubatuba,Sao Paulo,Brazil,-23.43,-45.07,America/Sao_Paulo,1656513650,2022-06-29 11:40,64.9,...,2022-06-30,22.7,10.8,15.2,Não,0,06:42 AM,05:26 PM,Sunny,12.2


In [ ]:
dataframe_previsoes_visao_geral = dataframe_previsoes_visao_geral.toPandas()

In [ ]:
dataframe_previsoes_visao_geral.to_csv('dataframe_previsoes_view_geral')

In [ ]:
# Criar DF da Tabela 1
# TODO

df_spark_tabela1 = spark.createDataFrame(df_forecast)

try:
    df_spark_tabela1.createTempView('view_dados_previsoes')
except:
    print('já existe esta view')

view_dados_previsoes = spark.sql("SELECT Cidade, CodigoCidade, Regiao, Pais, Latitude, Longitude, TemperaturaMax, TemperaturaMin, TemperaturaMed,VaiChover, ProbChuvaPercent, CondicaoDoTempo, NascerSol, PorSol, VelocidadeMaximaDoVento from view_dados_previsoes") 

In [ ]:
view_dados_previsoes = view_dados_previsoes.toPandas()

In [ ]:
view_dados_previsoes.to_csv('view_dados_previsoes')

In [ ]:
# Criar DF da Tabela 2
# TODO

df_spark_tabela2 = spark.createDataFrame(df_forecast)

try:
    df_spark_tabela2.createTempView('view_dias_chuvas')
except:
    print('já existe esta view')
    
view_dias_chuvas = spark.sql("SELECT Cidade, sum(case when VaiChover = 'Sim' then 1 else 0 end) as NaoVaiChover, sum(case when VaiChover = 'Não' then 1 else 0 end) as VaiChover, count(Data) as DiasMapeados from view_dias_chuvas group by Cidade")   

In [ ]:
view_dias_chuvas = view_dias_chuvas.toPandas()

In [ ]:
view_dias_chuvas.to_csv('view_dias_chuvas')